#**Assembly Workshop -- Analyze Movie Reviews**


##**Part One - Data Preprocessing**

##**Step 1: Import the required libraries**

Import the numpy and tensorflow libraries.

In [1]:
import numpy as np
import tensorflow as tf

##**Step 2: Import the dataset**

Our dataset comes from tf.keras.datasets.imdb, and has two functions: `get_word_index()` and `load_data()`.

Let's begin by loading our data into our testing and training sets

In [ ]:
data = tf.keras.datasets.imdb

(x_train, y_train), (x_test, y_test) = data.load_data()


##**Step 3: Importing our dictionary (map)**

`get_word_index() ` returns a dictionary of each word mapping to a certain integer

In [ ]:
word_to_int = data.get_word_index()

##**Step 4: Writing a function to decode the text**

First, `index_from` is set to 3. So we need to 'shift' our dictionary by 3 ints.

In [ ]:
word_to_int = {k : (v+3) for k,v in word_to_int.items()}

Moreover, we need to map some keys into ints for processing. Remember that when we call the `load_data()` function, the 'start' character is 1, out-of-vocabulary 'character' is 2, and 0 is for padding. 

In [ ]:
word_to_int['<PAD>'] = 0
word_to_int['<START>'] = 1
word_to_int['<UNK>'] = 2
word_to_int['<UNUSED>'] = 3

Next, we need to map each int to a word in order to decode the dataset.

In [ ]:
int_to_word = dict([(value, key) for (key, value) in word_to_int.items()])

Finally, we can write our decode function.

In [ ]:
def decode_text(text):
  return ' '.join([int_to_word.get(i, '?') for i in text])

##**Step 5: Padding our data**

Its important that all our data that's being fed into our model is of the same length, so we'll be padding data that is shorter than our specified length and trimming the data that is longer.

In [ ]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(sequences = x_train,
                                                        maxlen = 300,
                                                        padding = 'post',
                                                        value = word_to_int['<PAD>'])

x_test = tf.keras.preprocessing.sequence.pad_sequences(sequences = x_test,
                                                        maxlen = 300,
                                                        padding = 'post',
                                                        value = word_to_int['<PAD>'])

#**Part 2: Building the NN**

##**Step 1: Initialise the NN**

To initialise the NN, we need to find the Sequential class from the tensorflow keras library.

In [ ]:
model = tf.keras.models.Sequential()

##**Step 2: Adding the Embedding layer**

The embedding layer must be the first layer of our neural network. `input_dim` is going to be the number of words in our dictionary (check `load_data()`), and `output_dim` is going to be the dimensions of our vectors.

In [ ]:
model.add(tf.keras.layers.Embedding(input_dim = 88500, output_dim = 16))

##**Step 3: Adding the Global Average Pooling 1D layer**

This layer simply reduces the dimensions in our vectors to make it computationally less expensive to run.

In [ ]:
model.add(tf.keras.layers.GlobalAveragePooling1D())

##**Step 4: Adding the Dense layers**

Our first dense layer will have 32 neurons and will use the 'relu' activation function.

In [ ]:
model.add(tf.keras.layers.Dense(units = 32, activation = 'relu'))

Our second dense layer is also our output neuron, so it will only have 1 neuron and a 'sigmoid' activation function.

In [ ]:
model.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

#**Part 3: Training our model**

##**Step 1: Compiling our model**

We'll compile the network using the adam optimizer and the binary crossentropy loss function.

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

##**Step 2: Training the NN**

Using the .fit function, we will train the NN using a batch size of 1000 through 50 epochs. Moreover, we'll use 20% of our data as a validation set.

In [ ]:
model.fit(x_train, y_train, epochs = 50, batch_size = 1000, validation_split = 0.2)

#**Part 4: Testing the NN**

##**Step 1: Encoding our review**

We need to create a function to encode the review so that our model can read and process the data.

In [ ]:
def encode_review(text):
    encoded = []
    encoded.append(1)
    
    for word in text:
        if word.lower() in word_to_int:
            encoded.append(word_to_int[word.lower()])
        else:
            encoded.append(2)
    
    return encoded

##**Step 2: Reading and classifying the review**

Now, we can enter the file name and pass the data into our model to predict whether or not the reviewer liked or disliked the movie.

In [ ]:
filename = 'rev.txt'

#open text file and predict output  
with open(filename) as file:
    for line in file.readlines():
        #remove all unneccessary characters from the text file
        newLine = line.replace(',',"").replace('-', '').replace('.',"").replace('\"',"").replace('\'',"").replace(':',"").replace('(',"").replace(')',"").strip().split(' ')
        encode = encode_review(newLine)
        
        encode = tf.keras.preprocessing.sequence.pad_sequences([encode], maxlen = 300, padding='post',value = word_to_int['<PAD>'])
        
        predict = model.predict(encode)
        print(line)
        print(encode)
        print(predict)